In [5]:
import pandas as pd
df = pd.read_csv("Grocery_Inventory_and_Sales_Dataset.csv")
df.head()

,Product_ID,Product_Name,Catagory,Supplier_ID,Supplier_Name,Stock_Quantity,Reorder_Level,Reorder_Quantity,Unit_Price,Date_Received,Last_Order_Date,Expiration_Date,Warehouse_Location,Sales_Volume,Inventory_Turnover_Rate,Status
0,29-205-1132,Sushi Rice,Grains & Pulses,38-037-1699,Jaxnation,22,72,70,$4.50,8/16/2024,6/29/2024,9/19/2024,48 Del Sol Trail,32,19,Discontinued
1,40-681-9981,Arabica Coffee,Beverages,54-470-2479,Feedmix,45,77,2,$20.00,11/1/2024,5/29/2024,5/8/2024,36 3rd Place,85,1,Discontinued
2,06-955-3428,Black Rice,Grains & Pulses,54-031-2945,Vinder,30,38,83,$6.00,8/3/2024,6/10/2024,9/22/2024,3296 Walton Court,31,34,Backordered
3,71-594-6552,Long Grain Rice,Grains & Pulses,63-492-7603,Brightbean,12,59,62,$1.50,12/8/2024,2/19/2025,4/17/2024,3 Westerfield Crossing,95,99,Active
4,57-437-1828,Plum,Fruits & Vegetables,54-226-4308,Topicstorm,37,30,74,$4.00,7/3/2024,10/11/2024,10/5/2024,15068 Scoville Court,62,25,Backordered


In [6]:
date_columns = ["Date_Received", "Last_Order_Date", "Expiration_Date"]
for col in date_columns:
    df[col] = pd.to_datetime(df[col], errors='coerce')
df[["Date_Received", "Expiration_Date", "Last_Order_Date"]].head()


,Date_Received,Expiration_Date,Last_Order_Date
0,2024-08-16,2024-09-19,2024-06-29
1,2024-11-01,2024-05-08,2024-05-29
2,2024-08-03,2024-09-22,2024-06-10
3,2024-12-08,2024-04-17,2025-02-19
4,2024-07-03,2024-10-05,2024-10-11


In [7]:
df["days_to_expiry"] = (df["Expiration_Date"] - df["Date_Received"]).dt.days
df["days_to_expiry"].describe()


count    990.000000
mean       0.185859
std      150.125993
min     -365.000000
25%     -106.750000
50%       -1.500000
75%      100.750000
max      344.000000
Name: days_to_expiry, dtype: float64

In [8]:
df["Unit_Price"] = df["Unit_Price"].replace('[\$,]', '', regex=True).astype(float)
df["Unit_Price"].head()


0     4.5
1    20.0
2     6.0
3     1.5
4     4.0
Name: Unit_Price, dtype: float64

In [9]:
numerical_cols = ["Stock_Quantity", "Reorder_Level", "Reorder_Quantity",
                  "Unit_Price", "Sales_Volume", "Inventory_Turnover_Rate", "days_to_expiry"]

for col in numerical_cols:
    if df[col].isnull().any():
        df[col] = df[col].fillna(df[col].median())
df[numerical_cols].isnull().sum()


Stock_Quantity             0
Reorder_Level              0
Reorder_Quantity           0
Unit_Price                 0
Sales_Volume               0
Inventory_Turnover_Rate    0
days_to_expiry             0
dtype: int64

In [10]:
categorical_cols = ["Status", "Catagory", "Product_Name", "Warehouse_Location"]

for col in categorical_cols:
    if df[col].isnull().any():
        df[col] = df[col].fillna(df[col].mode()[0])
df[categorical_cols].isnull().sum()


Status                0
Catagory              0
Product_Name          0
Warehouse_Location    0
dtype: int64

In [11]:
df["Status_encoded"] = df["Status"].astype("category").cat.codes
df["Catagory_encoded"] = df["Catagory"].astype("category").cat.codes
df[["Status", "Status_encoded", "Catagory", "Catagory_encoded"]].head()


,Status,Status_encoded,Catagory,Catagory_encoded
0,Discontinued,2,Grains & Pulses,4
1,Discontinued,2,Beverages,1
2,Backordered,1,Grains & Pulses,4
3,Active,0,Grains & Pulses,4
4,Backordered,1,Fruits & Vegetables,3


In [12]:
drop_cols = ["Product_ID", "Product_Name", "Supplier_ID", "Supplier_Name",
             "Warehouse_Location", "Date_Received", "Expiration_Date",
             "Last_Order_Date", "Status", "Catagory"]

df_clean = df.drop(columns=drop_cols)
df_clean = df_clean[df_clean["days_to_expiry"] >= 0]
df_clean["days_to_expiry"].min()


0

In [13]:
def remove_outliers_iqr(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    return df[(df[column] >= lower) & (df[column] <= upper)]

outlier_columns = ["Unit_Price", "Sales_Volume", "Inventory_Turnover_Rate", "days_to_expiry"]
for col in outlier_columns:
    df_clean = remove_outliers_iqr(df_clean, col)
df_clean[outlier_columns].describe()


,Unit_Price,Sales_Volume,Inventory_Turnover_Rate,days_to_expiry
count,462.000000,462.000000,462.000000,462.000000
mean,4.493398,59.352814,50.712121,120.352814
std,2.619244,22.726302,28.953202,87.085610
min,0.200000,20.000000,1.000000,0.000000
25%,2.500000,41.000000,24.250000,49.000000
50%,4.000000,58.000000,52.000000,98.000000
75%,6.000000,80.000000,75.000000,189.000000
max,12.000000,100.000000,100.000000,344.000000


In [14]:
print(df_clean.isnull().sum())


Stock_Quantity             0
Reorder_Level              0
Reorder_Quantity           0
Unit_Price                 0
Sales_Volume               0
Inventory_Turnover_Rate    0
days_to_expiry             0
Status_encoded             0
Catagory_encoded           0
dtype: int64


In [15]:
df_clean.to_csv("final_cleaned_data.csv", index=False)


In [17]:
# data preprocessing ended here


In [18]:
# feature engineering code starts from here

In [19]:
# Feature: Is the product expiring in next 7 days?
df_clean["is_expiring_soon"] = df_clean["days_to_expiry"].apply(lambda x: 1 if x <= 7 else 0)
df_clean["stock_to_reorder_ratio"] = df_clean["Stock_Quantity"] / (df_clean["Reorder_Level"] + 1)
df_clean["turnover_per_day"] = df_clean["Inventory_Turnover_Rate"] / (df_clean["days_to_expiry"] + 1)
def demand_category(x):
    if x > 70:
        return "high"
    elif x > 40:
        return "medium"
    else:
        return "low"

df_clean["demand_level"] = df_clean["Sales_Volume"].apply(demand_category)
df_clean["demand_encoded"] = df_clean["demand_level"].astype("category").cat.codes
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
df_clean[["Unit_Price", "stock_to_reorder_ratio", "turnover_per_day"]] = scaler.fit_transform(
    df_clean[["Unit_Price", "stock_to_reorder_ratio", "turnover_per_day"]])
df_clean.head()



,Stock_Quantity,Reorder_Level,Reorder_Quantity,Unit_Price,Sales_Volume,Inventory_Turnover_Rate,days_to_expiry,Status_encoded,Catagory_encoded,is_expiring_soon,stock_to_reorder_ratio,turnover_per_day,demand_level,demand_encoded
0,22,72,70,0.364407,32,19,34,2,4,0,0.004153,0.007474,low,1
2,30,38,83,0.491525,31,34,50,1,4,0,0.014577,0.009193,low,1
4,37,30,74,0.322034,62,25,94,1,3,0,0.024030,0.003589,medium,2
6,96,52,16,0.194915,67,13,94,0,5,0,0.037794,0.001834,medium,2
7,44,90,17,0.194915,21,91,2,2,2,1,0.008212,0.421258,low,1


In [20]:
# meaning of the new features that are added
#is_expiring_soon	1 if days_to_expiry ≤ 7
# stock_to_reorder_ratio	Ratio of stock to reorder level
# . High ratio → zyada stock hai → price reduce karna pade
# . Low ratio → stock kam hai → price increase ho sakta
#turnover_per_day	Inventory turnover per day
# .High value → product fast bech raha hai
# .Low value → product slow move kar raha hai
#demand_level	High/Medium/Low demand (based on sales)
#demand_encoded	Encoded version of demand level



In [22]:
# Save the cleaned and feature-engineered data to CSV
df_clean.to_csv("cleaned_grocery_data.csv", index=False)


In [2]:
import os
os.getcwd()


'C:\\Users\\Diya arora'